In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *
from pathlib import Path
import pandas as pd

In [3]:
#path = Path('/home/ubuntu/data/autopsy/milestone/suicide_homicide_accident/no_preprocessing')
path = Path('/nfs/research/ejguill/data/autopsy/no_preprocessing/folds/tmp')


In [4]:
df = pd.read_csv(path/'data.csv')
#df.head()

In [5]:
#df.tail()

In [6]:
df['text'][1]

'\n.MICRO.\nHeart - no significant histopathology\n\nLungs - congestion, no significant histopathology\n\nKidneys - no significant histopathology\n\nLiver - no significant histopathology\n\nHippocampus - no significant histopathology\n.LOCATIONExtremities\n.INJURYBlunt injury.\n.INJURYDESC.\nHealing abrasions measuring from 0.1 to 0.2 cm are on the posterior right hand.\n \nA 0.2 x 1.1 cm healing abrasion is in the left inguinal fold. \n\n\n\n.LOCATIONHead\n.INJURYBlunt injury.\n.INJURYDESC.\nA 0.9 x 0.6 cm abrasion is on the right side of the chin. '

In [7]:
data_lm = TextLMDataBunch.from_csv(path, 'data.csv')

In [8]:
learn = language_model_learner(data_lm, qrnn=True, emb_sz=100, nh=200, nl=1)

In [9]:
learn.load_pretrained('/nfs/research/ejguill/data/pre-trained-models/qrnn/1-layer/models/qrnn_1.pth','/nfs/research/ejguill/data/pre-trained-models/qrnn/1-layer/tmp/itos.pkl')
learn.freeze()

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

In [ ]:
learn.fit_one_cycle(2, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save_encoder('fine_tuned_qrnn-1_tmp')

In [ ]:
learn.recorder.plot()

In [ ]:
bs=32

In [ ]:
data_clas = TextClasDataBunch.from_csv(path, 'data.csv', bs=bs)

In [ ]:
data_clas.show_batch()

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.5, qrnn=True, emb_sz=100, nh=200, nl=1)
learn.load_encoder('fine_tuned_qrnn-1_tmp')

learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(20, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
from fastai.vision import *

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(4,4), dpi=120)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data_clas.valid_ds)==len(losses)==len(idxs)

In [ ]:
loss = learn.recorder.losses

In [ ]:
len(loss)

In [ ]:
len(learn.recorder.val_losses)